In [1]:
#load env
from dotenv import load_dotenv
load_dotenv()

True

### Trainning Model

In [2]:
import os
os.environ['AWS_SECRET_ACCESS_KEY']

'SqcfFREIjVriLN+F3gFcavLZFypXUG98mV+LIFbH'

In [3]:
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import boto3

sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

bucket = os.environ['BUCKET_NAME']

iris = load_iris()
iris.keys()

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\voanh\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\voanh\AppData\Local\sagemaker\sagemaker\config.yaml


dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
trainX, testX, trainY, testY = train_test_split(
    iris["data"], iris["target"], test_size=0.25
)
print("[INFO] train shape: {}".format(trainX.shape))
print("[INFO] test shape: {}".format(testX.shape))

[INFO] train shape: (112, 4)
[INFO] test shape: (38, 4)


In [13]:
train = pd.DataFrame(trainX, columns=iris["feature_names"])
train["target"] = pd.Series(trainY)

train.to_csv("iris_train.csv", index=False, header=True)

test = pd.DataFrame(testX, columns=iris["feature_names"])
test["target"] = pd.Series(testY)

test.to_csv("iris_test.csv", index=False, header=True)

In [14]:
sk_prefix="sagemarker/examples/sk_iris"
train_path = sess.upload_data(
    path="iris_train.csv", bucket=bucket, key_prefix=sk_prefix
)
test_path = sess.upload_data(
    path="iris_test.csv", bucket=bucket, key_prefix=sk_prefix
)

In [15]:
%%writefile sklearn_script.py
import argparse
import joblib
import numpy as np
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

parser = argparse.ArgumentParser()

parser.add_argument("--train-test-ratio", type=float, default=0.25)
parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))

args = parser.parse_args()

print("[INFO] Start training...")

train = pd.read_csv(os.path.join(args.train, "iris_train.csv"))
test = pd.read_csv(os.path.join(args.test, "iris_test.csv"))

X_train, X_test, y_train, y_test = train_test_split(
    train.drop(["target"], axis=1), train["target"], test_size=args.train_test_ratio
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train, y_train)

joblib.dump(model, os.path.join(args.model_dir, "model.joblib"))
print(f"[INFO] Model dumped to {os.path.join(args.model_dir, 'model.joblib')}")
y_pred = model.predict(X_test)
print(f"[INFO] Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"[INFO] Log loss: {log_loss(y_test, model.predict_proba(X_test))}")


Overwriting sklearn_script.py


In [16]:
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
    entry_point="sklearn_script.py",
    role=os.environ["AWS_ARN_ROLE"],
    train_instance_count=1,
    train_instance_type="ml.m5.4xlarge",
    framework_version="0.23-1",
    base_job_name="sklearn-script",
    hyperparameters={},
    max_run=300, # 5 minutes
    max_wait=600, # 10 minutes
    use_spot_instances=True
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\voanh\AppData\Local\sagemaker\sagemaker\config.yaml


In [17]:
sklearn_estimator.fit({"train": train_path, "test": test_path}, wait=True)

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: sklearn-script-2023-12-03-11-14-47-136


2023-12-03 11:15:04 Starting - Starting the training job...
2023-12-03 11:15:19 Starting - Preparing the instances for training......
2023-12-03 11:16:26 Downloading - Downloading input data...
2023-12-03 11:17:02 Training - Training image download completed. Training in progress..2023-12-03 11:17:19,088 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-12-03 11:17:19,092 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-03 11:17:19,130 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-12-03 11:17:19,288 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-03 11:17:19,299 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-03 11:17:19,311 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-03 11:17:19,320 sagemaker-training-toolkit INFO     Invoking user scrip

### Deploy models

In [18]:
from sagemaker.sklearn.model import SKLearnModel

model_name = "sklearn-script-model"
model = SKLearnModel(
    model_data=sklearn_estimator.model_data,
    role=os.environ["AWS_ARN_ROLE"],
    entry_point="sklearn_script.py",
    framework_version="0.23-1",
    name=model_name
)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\voanh\AppData\Local\sagemaker\sagemaker\config.yaml


In [ ]:
enpoint_name = "sklearn-script-endpoint"
preidctor = model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge", endpoint_name=enpoint_name)

In [ ]:
#remove
sm_boto3.delete_endpoint(EndpointName=enpoint_name)